In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
from seaborn import color_palette
import numpy as np
from util import load_results_from_dir

sns.set_palette('Set1')
plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        # "font.serif": ["Computer Modern Roman", "Times New Roman"] + plt.rcParams["font.serif"],
        "text.latex.preamble": r"\usepackage[T1]{fontenc} \usepackage[utf8]{inputenc} \usepackage{lmodern}",
    }
)
figsize=(4,2)
dpi = 120
parameters = ['scheduler', 'N_w', 'N_d', 'N_s', 'W_hat', 'pi', 'sigma', 'epsilon', 'int_feas_tol', 'v', 'r_charge', 'r_deplete', 'B_min', 'B_max', 'B_start']

In [ ]:
rootdir = "../out/villalvernia.elrond/charging_station_utility"
df = load_results_from_dir(rootdir)
df['experiment_type'] = None
df.loc[(df.scheduler == 'naivescheduler') & (df.N_w == 17), 'experiment_type'] = 'naive_coarse'
df.loc[(df.scheduler == 'naivescheduler') & (df.N_w == 74), 'experiment_type'] = 'naive_fine'
df.loc[(df.scheduler == 'milpscheduler') & (df.N_w == 17), 'experiment_type'] = 'milp_coarse'
df.loc[(df.scheduler == 'milpscheduler') & (df.N_w == 74), 'experiment_type'] = 'milp_fine'

# filter prior results that are not relevant
df = df.dropna(subset=['experiment_type'])

# add extra columns
df['charge_ratio'] = np.round(df.r_deplete / df.r_charge, 1)
df = df.drop_duplicates(['charge_ratio', 'experiment_type', 'N_w', 'N_s'])

In [ ]:
print(f"Size of dataset: {df.shape[0]:,}\n")
display(df[['experiment_type', 'N_w', 'N_s']].value_counts().reset_index().rename(columns={0: '# datapoints'}))

# Coarse

In [ ]:
df_coarse = df[lambda x: x.experiment_type.isin(['milp_coarse', 'naive_coarse'])]

In [ ]:
label_translation = {
    'milp_coarse': 'Optimal',
    'naive_coarse': 'Naive',
}

In [ ]:
ymax = df_coarse.execution_time.max() + 100

for i, N_s in enumerate([1, 2, 3]):
    _, ax = plt.subplots(figsize=figsize, dpi=dpi)
    
    hue_order = ['milp_coarse', 'naive_coarse']
    sns.barplot(data=df_coarse[lambda x: x.N_s == N_s], x='charge_ratio', y='execution_time', hue='experiment_type', hue_order=hue_order, edgecolor='black', alpha=0.7, ax=ax)

    handles, labels = ax.get_legend_handles_labels()
    new_labels = [label_translation[i] for i in labels]
    leg = ax.legend(handles, new_labels, frameon=False)
    if i != 0:
        leg.remove()

    ax.set_xlabel("Charge ratio " + r"$(r^{-} / r^{+})$")
    ax.set_ylabel("Execution time (s)")
    ax.set_ylim([0, ymax])
    plt.grid(axis='y')

    plt.savefig(f"../out/figures/charging_station_utility/N_s{N_s}.execution_time.coarse.bar.pdf", bbox_inches='tight')
    plt.show()

In [ ]:
for i, N_s in enumerate([1, 2, 3]):
    _, ax = plt.subplots(figsize=figsize, dpi=dpi)
    
    hue_order = ['milp_coarse', 'naive_coarse']
    sns.barplot(data=df_coarse[lambda x: x.N_s == N_s], x='charge_ratio', y='utilization', hue='experiment_type', hue_order=hue_order, edgecolor='black', alpha=0.7, ax=ax)

    handles, labels = ax.get_legend_handles_labels()
    new_labels = [label_translation[i] for i in labels]
    leg = ax.legend(handles, new_labels, frameon=False)
    if i != 0:
        leg.remove()


    ax.set_xlabel("Charge ratio " + r"$(r^{-} / r^{+})$")
    ax.set_ylabel("Utilization")
    plt.grid(axis='y')
    plt.ylim([0, 1])

    plt.savefig(f"../out/figures/charging_station_utility/N_s{N_s}.utilization.coarse.bar.pdf", bbox_inches='tight')
    plt.show()

In [ ]:
for i, N_s in enumerate([1, 2, 3]):
    _, ax = plt.subplots(figsize=figsize, dpi=dpi)
    
    hue_order = ['milp_coarse', 'naive_coarse']
    sns.barplot(data=df_coarse[lambda x: x.N_s == N_s], x='charge_ratio', y='frac_waited', hue='experiment_type', hue_order=hue_order, edgecolor='black', alpha=0.7, ax=ax)

    handles, labels = ax.get_legend_handles_labels()
    new_labels = [label_translation[i] for i in labels]
    leg = ax.legend(handles, new_labels, frameon=False)
    if i != 0:
        leg.remove()


    ax.set_xlabel("Charge ratio " + r"$(r^{-} / r^{+})$")
    ax.set_ylabel("Fraction of time\nspent waiting")
    plt.grid(axis='y')
    plt.ylim([0, 1])

    plt.savefig(f"../out/figures/charging_station_utility/N_s{N_s}.frac_waited.coarse.bar.pdf", bbox_inches='tight')
    plt.show()

# Fine-grained

In [ ]:
df_fine = df[lambda x: x.experiment_type.isin(['milp_fine', 'naive_fine'])]

In [ ]:
label_translation = {
    'milp_fine': '$\sigma=8$',
    'naive_fine': 'Naive',
}

In [ ]:
cp = color_palette()
palette = [cp[2], cp[1]]

ymax = df_fine.execution_time.max() + 300

for i, N_s in enumerate([1, 2, 3]):
    _, ax = plt.subplots(figsize=figsize, dpi=dpi)

    hue_order = ['milp_fine', 'naive_fine']
    sns.barplot(data=df_fine[lambda x: x.N_s == N_s], x='charge_ratio', y='execution_time', hue='experiment_type', hue_order=hue_order, edgecolor='black', alpha=0.7, ax=ax, palette=palette)

    handles, labels = ax.get_legend_handles_labels()
    new_labels = [label_translation[i] for i in labels]
    leg = ax.legend(handles, new_labels, frameon=False)
    if i != 0:
        leg.remove()

    ax.set_xlabel("Charge ratio " + r"$(r^{-} / r^{+})$")
    ax.set_ylabel("Execution time (s)")
    ax.set_ylim([0, ymax])
    plt.grid(axis='y')

    plt.savefig(f"../out/figures/charging_station_utility/N_s{N_s}.execution_time.fine.bar.pdf", bbox_inches='tight')
    plt.show()

In [ ]:
cp = color_palette()
palette = [cp[2], cp[1]]

ymax = df_fine.execution_time.max() + 100

for i, N_s in enumerate([1, 2, 3]):
    _, ax = plt.subplots(figsize=figsize, dpi=dpi)

    hue_order = ['milp_fine', 'naive_fine']
    sns.barplot(data=df_fine[lambda x: x.N_s == N_s], x='charge_ratio', y='utilization', hue='experiment_type', hue_order=hue_order, edgecolor='black', alpha=0.7, ax=ax, palette=palette)

    handles, labels = ax.get_legend_handles_labels()
    new_labels = [label_translation[i] for i in labels]
    leg = ax.legend(handles, new_labels, frameon=False)
    if i != 0:
        leg.remove()

    ax.set_xlabel("Charge ratio " + r"$(r^{-} / r^{+})$")
    ax.set_ylabel("Utilization")
    ax.set_ylim([0, 1])
    plt.grid(axis='y')

    plt.savefig(f"../out/figures/charging_station_utility/N_s{N_s}.utilization.fine.bar.pdf", bbox_inches='tight')
    plt.show()

In [ ]:
cp = color_palette()
palette = [cp[2], cp[1]]

ymax = df_fine.execution_time.max() + 100

for i, N_s in enumerate([1, 2, 3]):
    _, ax = plt.subplots(figsize=figsize, dpi=dpi)

    hue_order = ['milp_fine', 'naive_fine']
    sns.barplot(data=df_fine[lambda x: x.N_s == N_s], x='charge_ratio', y='frac_waited', hue='experiment_type', hue_order=hue_order, edgecolor='black', alpha=0.7, ax=ax, palette=palette)

    handles, labels = ax.get_legend_handles_labels()
    new_labels = [label_translation[i] for i in labels]
    leg = ax.legend(handles, new_labels, frameon=False)
    if i != 0:
        leg.remove()

    ax.set_xlabel("Charge ratio " + r"$(r^{-} / r^{+})$")
    ax.set_ylabel("Fraction of time\nspent waiting")
    ax.set_ylim([0, 1])
    plt.grid(axis='y')

    plt.savefig(f"../out/figures/charging_station_utility/N_s{N_s}.frac_waited.fine.bar.pdf", bbox_inches='tight')
    plt.show()